In [1]:
%pip install -U chromadb==0.3.22 langchain==0.0.164 transformers==4.29.0 accelerate==0.19.0

  Using cached langchain-0.0.164-py3-none-any.whl.metadata (10 kB)
  Using cached transformers-4.29.0-py3-none-any.whl.metadata (111 kB)
  Using cached accelerate-0.19.0-py3-none-any.whl (219 kB)
  Using cached dataclasses_json-0.5.14-py3-none-any.whl.metadata (22 kB)
  Using cached numexpr-2.9.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.9 kB)
  Using cached openapi_schema_pydantic-1.2.4-py3-none-any.whl (90 kB)
  Using cached pydantic-1.10.14-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (150 kB)
  Using cached tokenizers-0.13.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
Using cached langchain-0.0.164-py3-none-any.whl (788 kB)
Using cached transformers-4.29.0-py3-none-any.whl (7.1 MB)
Using cached dataclasses_json-0.5.14-py3-none-any.whl (26 kB)
Using cached numexpr-2.9.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (374 kB)
Using cached pydantic-1.10.14-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_

In [1]:
import pandas as pd
import numpy as np

In [2]:
#user_docs = pd.read_json('/content/drive/MyDrive/Colab Notebooks/RAG/userdocuments.json')
#user_docs = pd.read_json('userdocuments.json')
#user_docs.drop(columns='_id', inplace=True)
#user_docs.head()

In [3]:
with open('payments.csv', 'r') as file:
    lines = file.readlines()[1:]

# Crear un DataFrame con una columna llamada 'content'
user_docs = pd.DataFrame({'userId':'644ae4cf97c530c581cb0f7c', 'content': lines})
#user_docs = pd.concat([user_docs, user_fees], ignore_index=True).reset_index(drop=True)
user_docs.head()

,userId,content
0,644ae4cf97c530c581cb0f7c,Este es un pago de 10550 COP para pagar Gas. E...
1,644ae4cf97c530c581cb0f7c,Este es un pago de 1620000 COP para pagar Arri...
2,644ae4cf97c530c581cb0f7c,Este es un pago de 133020 COP para pagar Luz. ...
3,644ae4cf97c530c581cb0f7c,Este es un pago de 79700 COP para pagar Intern...
4,644ae4cf97c530c581cb0f7c,Este es un pago de 36480 COP para pagar Gas. E...


In [9]:
from langchain.embeddings import HuggingFaceEmbeddings

#hf_embed = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
hf_embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [10]:
from langchain.docstore.document import Document
from langchain.vectorstores import Chroma

documents = [Document(page_content=text, metadata={"source": source}) for text, source in zip(user_docs.content, user_docs.userId)]

#vector_db_path = '/content/drive/MyDrive/Colab Notebooks/RAG/Vector DB'
vector_db_path = './vectordb'
db = Chroma.from_documents(collection_name="user_docs", documents = documents, embedding=hf_embed, persist_directory=vector_db_path)

#db.similarity_search('dummy')

db.persist()

print(f"Document embedding saved under {vector_db_path}")


Using embedded DuckDB with persistence: data will be stored in: ./vectordb


Document embedding saved under ./vectordb


In [12]:

def get_similar_docs(question, similar_doc_count):

  return db.similarity_search(question, k=similar_doc_count)

for doc in get_similar_docs('pago de arriendo', 2):
  print(doc.page_content)


Este es un pago de 2620200 COP para pagar Arriendo. Es un pago recurrente para pagar los días 3. La forma de hacer este pago es Entrar a Vélez Osorio página y pago por PSE ingresando está CC 1020839613. Pago realizado el 3 de Noviembre de 2023.

Este es un pago de 2817700 COP para pagar Arriendo. Es un pago recurrente para pagar los días 3. La forma de hacer este pago es Entrar a Vélez Osorio página y pago por PSE ingresando está CC 1020839613. Pago realizado el 2 de Agosto de 2023.

